<a href="https://colab.research.google.com/github/zzzsheep/NFL-Predictor-/blob/main/NFL_predictor_V1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
#Install Packages
!pip install nfl_data_py
!pip install pandas
!pip install os
!pip install matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.3 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [3]:
import nfl_data_py as nfl
import pandas as pd
import os
import urllib.request
import matplotlib.pyplot as plt

**Imports play-by-play data**

In [4]:
pbp_data = nfl.import_pbp_data([2021, 2022, 2023], downcast=True)

2021 done.
2022 done.
2023 done.
Downcasting floats.


**ALL functions used**

In [14]:
# Converts player name to match abbreviation name in dataset
def parse_name(player_name):
  parts = player_name.split()
  if len(parts) < 2:
    raise ValueError("No valid first or last name")
  first_init = parts[0][0]
  last_name = ' '.join(parts[1:])
  abbr_name = first_init + '.' + last_name
  return abbr_name

# Determine the correct column to filter based on the player’s position
def player_pos(position):
  if position == 'qb':
      return 'passer'
  elif position == 'rusher':
      return 'rusher'
  elif position == 'receiver':
      return'receiver'
  else:
      raise ValueError("Invalid position entered! Choose from passer, rusher, or receiver.")

# Calculates the average total yards per game
def average_calc(player_season, data_types=None):
  if data_types == 'current':
      total_games = player_season['games'].sum()
  else:
      total_games = player_season['game_id'].nunique()
  total_yards = player_season['passing_yards'].sum()

  print("total games played: " , total_games)
  print("total passing yards: ", total_yards)
  if total_games > 0:
    return round(total_yards / total_games, 2)   # Calculate average with total games and yards
  else:
    return 0  # Handle case where no games are played

# Displays data from player, has parameter for which season
def player_stats_summary(player_season):
  player_stats_summary = player_season.agg({
    'passing_yards': 'sum',
    'pass_touchdown': 'sum',
    'interception': 'sum'

})
  print(player_stats_summary)

# Gets player's player id by using roster import
def player_info(player_name):
  rosters = nfl.import_seasonal_rosters([2024], ['player_id', 'player_name', 'week'])
  player_info = rosters[rosters['player_name'].str.contains(player_name, case=False, na=False)]
  if not player_info.empty:
    player_id = player_info['player_id'].iloc[0]
    return player_id
  else:
    raise ValueError("player_id not found")

#pbp_data.columns.values.     <--shows all variables
#pbp_data.head(10)


**INPUT FROM USER**

In [15]:
test = False

if test:
  player_name = "Brock Purdy"
  abbr_player_name = "B.Purdy"
  oppo_team = "SEA"
  current_year = 2024
  last_year = current_year - 1
  column = player_pos('qb')
  game_type = "REG"
else:
  player_name = input("input of player name: ")
  abbr_player_name = parse_name(player_name)
  oppo_team = input("opposing team abbr: ")
  current_year = int(input("current season: "))
  column = player_pos(input("Enter player position (qb, rusher, receiver): ").strip().lower() )  # Ask for player position to decide which column to filter
  last_year = current_year - 1
  game_type = input("Choose which game type: (ALL, REG, POST) ")


#rint(player_current_season_stats.head())
# Display the filtered player's current season stats against the opponent
#print(player_current_season_stats.head())


input of player name: Lamar Jackson
opposing team abbr: DAL
current season: 2024
Enter player position (qb, rusher, receiver): qb
Choose which game type: (ALL, REG, POST) REG


TEAM PLAY DATA LAST 2-3 YEARS

In [16]:
# Filter play-by-play data for the player, opponent team, and current season
player_vs_oppo_stats = pbp_data[
    (pbp_data[column] == abbr_player_name) &           # Filter by the correct player column
    (pbp_data['defteam'] == oppo_team.upper()) &
    (pbp_data['season_type'] == game_type)  # Filter by opponent team
]
avg_passing_yards_vs_opp = average_calc(player_vs_oppo_stats)
print("average passing yards vs OPP: ", avg_passing_yards_vs_opp)

total games played:  0
total passing yards:  0.0
average passing yards vs OPP:  0


**PLAYER CURRENT SEASON STATS**

In [17]:
pbp_data_seasonal = nfl.import_seasonal_data([current_year], 'REG')
player_current_season_stats = pbp_data_seasonal[
    (pbp_data_seasonal['player_id'] == player_info(player_name) ) &
    (pbp_data_seasonal['season_type'] == game_type)
]      #Filter by season game type

avg_passing_yard_cur = average_calc(player_current_season_stats, 'current')
print(avg_passing_yard_cur)


total games played:  2
total passing yards:  520.0
260.0


**PLAYER LAST SEASON STATS**

In [18]:
player_last_season_stats = pbp_data[
    (pbp_data[column] == abbr_player_name) &           # Filter by the correct player column # Filter by opponent team
    (pbp_data['season'] == last_year)&
     (pbp_data['season_type'] == game_type)     # Filter by current season
]
avg_passing_yard_last = average_calc(player_last_season_stats)
print(avg_passing_yard_last)

total games played:  16
total passing yards:  3678.0
229.88


**CALCULATE HOW MANY WEEKS WE ARE IN THE SEASON**

In [11]:
# Get weekly data
pbp_data_weekly = nfl.import_weekly_data([2024], downcast=True)


current_week = pbp_data_weekly[pbp_data_weekly['season_type'] == game_type]['week'].max()
print("Current week in " + game_type + " season:", current_week)

Downcasting floats.
Current week in REG season: 2


**GET NUMBER OF GAMES PLAYER PLAYED CURRENT SEASON**

In [12]:
current_games = player_current_season_stats['games'].sum()

**CALCULATIONS FOR FORMULA**

In [13]:
percentage_tweaker = 1.1765
current_severity = 0.2 + (current_games * percentage_tweaker)/100
last_severity = 0.4 - (current_games * percentage_tweaker)/100
team_played_severity = 0.4
final_calculated_projection = avg_passing_yard_cur * current_severity + avg_passing_yard_last * last_severity +  avg_passing_yards_vs_opp * team_played_severity
print("final calculated projection: ", round(final_calculated_projection,2))

final calculated projection:  268.04


**QUARTERBACK** STAT

In [ ]:
qb_values = ['season', 'passer_id','passer', 'pass', 'passing_yards',
            'complete_pass', 'interception',
            'sack', 'yards_gained', 'pass_touchdown', 'touchdown']

groupby_values = ['season',  'passer_id','passer']

# group by and aggregate by sum
qb_pbp_data = (pbp_data
         .loc[:, qb_values]
         .groupby(groupby_values,
                  as_index=False)
         .sum())
pbp_data.head(5)
qb_pbp_data.sample(10)

In [ ]:
import random
import time

seasons = [str(season) for season in range (2024)]

player_abbrs = []
#create dataframe
nfl_df = pd.DataFrame()

#looping through the games
for season in seasons:
  for players in player_abbrs:
    url = "https://www.pro-football-reference.com/teams"

In [ ]:
pbp_fei = nfl.import_seasonal_rosters([2024])

# Load play-by-play data for the 2024 season
pbp_data_2024 = nfl.import_weekly_data([2024], downcast=True)
pbp_data_seasonal = nfl.import_seasonal_data([current_year], 'ALL')
print(pbp_data_seasonal.columns)
print(pbp_data_seasonal.sample(10))
# Check the first few rows to confirm you're getting 2024 season data
player_stats_summary(pbp_fei)
pbp_data_REG = nfl.import_seasonal_data([2024], 'REG')
#print(pbp_data_REG.head())